In [ ]:
%cd /data/ajay/contracode
%pwd
import pandas as pd
import plotnine as p9
%matplotlib inline
import jsonlines
import random
from pprint import pprint
from tqdm.auto import tqdm
import pickle
from collections import defaultdict, Counter

In [ ]:
def load_data(fname):
    with jsonlines.open(fname) as reader:
        return list(reader)
data = load_data('data/codesearchnet_javascript/javascript_train_supervised.jsonl')

def extract_fields(fields, gen):
    for item in gen:
        yield 

def parse_data(key, data):
    matches = filter(lambda x: key.lower() == x['func_name'].lower(), data)
    extracted_matches = map(lambda item: {k: v for k, v in item.items() if k in ['func_name', 'code']}, matches)
    return list(extracted_matches)

matches = defaultdict(list)
for x in tqdm(data):
    matches[x['func_name'].lower()].append(x['code'])

match_counts = Counter()
for k, v in matches.items():
    match_counts[k] += len(v)

In [ ]:
match_counts.most_common(25)

In [ ]:
%time dataset_indexes = list(enumerate(x[1] for x in match_counts.most_common(20000)))
%time df = pd.DataFrame(dataset_indexes, columns=['Method', 'Frequency'])
(p9.ggplot(df, p9.aes(x="Method", y="Frequency")) + p9.geom_bar(stat = "identity"))

In [ ]:
top_methods = set(x[0] for x in match_counts.most_common(500))
df_data = []
for method in tqdm(top_methods):
    df_data.extend(parse_data(method, data))
grouped_methods_df = pd.DataFrame(df_data)

In [ ]:
grouped_methods_df.to_pickle('data/tsne/supervised_top500_groups_df.pickle') 
grouped_methods_df

# Pack augmented dataset

In [ ]:
with open('/dev/shm/javascript_augmented.pickle', 'rb') as f:
    dataset = pickle.loads(f.read())

In [ ]:
import modin.pandas as pd
df = pd.DataFrame(dataset)

In [ ]:
import pandas as pd
import random
shuffled_list = list(range(len(dataset)))
random.shuffle(shuffled_list)
sample_10k = [dataset[i] for i in shuffled_list[:10000]]
sample_df = pd.DataFrame([dict(text=list(x)) for x in sample_10k], columns=['text'])
sample_df.to_pickle('/data/ajay/contracode/data/codesearchnet_javascript/augmented_10k_subset.pickle')

In [ ]:
pd.read_pickle('/data/ajay/contracode/data/codesearchnet_javascript/augmented_10k_subset.pickle')

In [ ]:
moco_path = "data/tsne/moco_embed.pickle"
!ls -lah {moco_path}
with open(moco_path, 'rb') as f:
    x = f.read()

In [ ]:
data = pickle.loads(x)

In [ ]:
data[0][0]